In [1]:
#import libraries.
import numpy as np
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
from tensorflow.python.keras.utils.data_utils import Sequence
from keras.models import Sequential
from keras.layers import Dense, LeakyReLU
from keras.wrappers.scikit_learn import KerasRegressor
import keras
from keras import regularizers, optimizers
import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from keras.initializers import glorot_normal
import sounddevice as sd
import tensorflow as tf
from mir_eval import separation 
from pystoi.stoi import stoi 
import h5py
from keras.callbacks import LearningRateScheduler, Callback, ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.layers import Activation
import math
from sklearn.utils import shuffle
from sklearn import preprocessing
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
#import libraries.
# import matplotlib.pyplot as plt
# from tabulate import tabulate
import time
import os
import librosa
from librosa.core import stft, istft
import time
import pickle
from keras import backend as K
from tensorflow.keras.models import load_model
# from keras.models import load_model

In [3]:
def reconstruct(wave,angle):
    recon1 = wave*np.cos(angle)+wave*np.sin(angle)*1j
#     recon = np.sqrt(np.power(10, wave))
#     recon1 = recon*np.cos(angle)+recon*np.sin(angle)*1j
    recon = librosa.core.istft((recon1.T), hop_length=512, win_length=2048, window='hann')
    return recon

In [4]:
data_series = str(1)

In [41]:
h5f = h5py.File('TSP/Organized/concatenated/Train_TSP_mixed_FFT'  + data_series + '.hdf5','r')
X = h5f['Train_TSP_mixed_FFT'  + data_series ][:,0:100]
h5f.close()

In [43]:
h5f = h5py.File('TSP/Organized/concatenated/Train_TSP_clean_FFTs'  + data_series + '.hdf5','r')
y = h5f['Train_TSP_clean_FFTs'  + data_series ][:,0:100]
h5f.close()

In [36]:
h5f = h5py.File('TSP/Organized/concatenated/Valid_TSP_mixed_FFT0.hdf5','r')
X_valid = h5f['Valid_TSP_mixed_FFT0'][:]
h5f.close()

In [37]:
h5f = h5py.File('TSP/Organized/concatenated/Valid_TSP_clean_FFTs0.hdf5','r')
y_valid = h5f['Valid_TSP_clean_FFTs0' ][:]
h5f.close()

In [38]:
X = preprocessing.normalize(X, norm='l2', axis=0, copy=True)
X_valid = preprocessing.normalize(X_valid, norm='l2', axis=0, copy=True)

In [44]:
a = y[0:1025]
b = y[1025:]
Mask = a/(a+b)
Mask = np.concatenate((Mask,b/(a+b)), axis = 0)

In [45]:
a = y_valid[0:1025]
b = y_valid[1025:]
Mask_valid = a/(a+b)
Mask_valid = np.concatenate((Mask_valid,b/(a+b)), axis = 0)

In [11]:
y = 0

In [47]:
sgd = optimizers.SGD(lr=0.0001,decay = 0.00001)

In [24]:
X_dim= 1025
y_dim = 2050
h_A = [1025, 1025, 1025]
def DNN_A():
    # create model
    model = Sequential()
    model.add(Dense(h_A[0], input_dim = X_dim, kernel_initializer='glorot_normal'))
#     keras.layers.Dropout(0.8, noise_shape=None, seed=None)
    model.add(BatchNormalization())
    model.add(Activation('sigmoid'))
#     model.add(LeakyReLU(alpha=0.1))

    model.add(Dense(h_A[1], kernel_initializer='glorot_normal'))
#     keras.layers.Dropout(0.8, noise_shape=None, seed=None)
    model.add(BatchNormalization())
    model.add(Activation('sigmoid'))
#     model.add(LeakyReLU(alpha=0.1))

    model.add(Dense(h_A[2], kernel_initializer='glorot_normal'))
    keras.layers.Dropout(0.8, noise_shape=None, seed=None)
    model.add(BatchNormalization())
    model.add(Activation('sigmoid'))
#     model.add(LeakyReLU(alpha=0.1))
    
    model.add(Dense(y_dim, kernel_initializer='glorot_normal'))
#     keras.layers.Dropout(0.8, noise_shape=None, seed=None)
    model.add(BatchNormalization())
    model.add(Activation('sigmoid'))
#     model.add(LeakyReLU(alpha=0.1))

    # Compile model
    model.compile(loss='mse', optimizer='adam')
    return model

In [48]:
# create model
X_dim= 1025
y_dim = 2050
h_A = [1025, 1025, 1025]
estimator_A = Sequential()
estimator_A.add(Dense(h_A[0], input_dim = X_dim, kernel_initializer='glorot_normal'))
#     keras.layers.Dropout(0.8, noise_shape=None, seed=None)
estimator_A.add(BatchNormalization())
estimator_A.add(Activation('sigmoid'))
#     model.add(LeakyReLU(alpha=0.1))

estimator_A.add(Dense(h_A[1], kernel_initializer='glorot_normal'))
#     keras.layers.Dropout(0.8, noise_shape=None, seed=None)
estimator_A.add(BatchNormalization())
estimator_A.add(Activation('sigmoid'))
#     model.add(LeakyReLU(alpha=0.1))

estimator_A.add(Dense(h_A[2], kernel_initializer='glorot_normal'))
# keras.layers.Dropout(0.8, noise_shape=None, seed=None)
estimator_A.add(BatchNormalization())
estimator_A.add(Activation('sigmoid'))
#     model.add(LeakyReLU(alpha=0.1))

estimator_A.add(Dense(y_dim, kernel_initializer='glorot_normal'))
#     keras.layers.Dropout(0.8, noise_shape=None, seed=None)
estimator_A.add(BatchNormalization())
estimator_A.add(Activation('sigmoid'))
#     model.add(LeakyReLU(alpha=0.1))

# Compile model
estimator_A.compile(loss='mse', optimizer='adam')

In [49]:
class SGDLearningRateTracker(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print('LR= ', K.eval(self.model.optimizer.lr))
        _lr = tf.to_float(self.model.optimizer.lr, name='ToFloat')
        _decay = tf.to_float(self.model.optimizer.decay, name='ToFloat')
        _iter = tf.to_float(self.model.optimizer.iterations, name='ToFloat')
        lr = K.eval(_lr * (1. / (1. + _decay * _iter)))
        print(' - LR: {:.6f}\n'.format(lr))
  
model_saving_filepath = "Models/Two_stage/adam_SBS/saved-model-{epoch:03d}-{loss:.4f}.hdf5"
model_saving_checkpoint = ModelCheckpoint(model_saving_filepath, monitor='loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)


In [14]:
min_delta = 0.0001

stopper = EarlyStopping(monitor='val_loss',min_delta=min_delta,patience=2) 
class OptimizerChanger(EarlyStopping):

    def __init__(self, on_train_end, **kwargs):

        self.do_on_train_end = on_train_end
        super(OptimizerChanger,self).__init__(**kwargs)

    def on_train_end(self, logs=None):
        super(OptimizerChanger,self).on_train_end(self,logs)
        self.do_on_train_end()
        
def do_after_training():

    #warining, this creates a new optimizer and,
    #at the beginning, it might give you a worse training performance than before
    model.compile(optimizer = 'SGD', loss='MSE')
#     model.fit(X.T, Mask.T, shuffle = True, epochs = 10, batch_size=10,  steps_per_epoch=None,)
    
changer = OptimizerChanger(on_train_end= do_after_training, 
                           monitor='val_loss',
                           min_delta=min_delta,
                           patience=2)

# model.fit(X.T, Mask.T, shuffle = True, epochs = 10, batch_size=10,  steps_per_epoch=None, callbacks = [changer])

In [50]:
# # seed = 1
# # np.random.seed(seed)
# print("^^^^^^^^^^^^^^^^^^^^^ Training with 5-fold on mixed0^^^^^^^^^^^^^^^^^^^^^")
batch_size = 10
tic = time.time()
# estimator_A = KerasRegressor(build_fn=DNN_A, epochs = 10, batch_size=batch_size, shuffle = True, verbose=1)
# kfold = KFold(n_splits=2, random_state=None)
# kfold = KFold(n_splits=5)
# results = cross_val_score(estimator_A, X.T, y.T, cv=kfold)
# print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))
# toc_fold = time.time()

print("^^^^^^^^^^^^^^^^^^^^^ Training on mixed5^^^^^^^^^^^^^^^^^^^^^")
# two_opt_filepath="half_trained.hdf5"
# two_opt_checkpoint = ModelCheckpoint(two_opt_filepath, monitor = 'loss', verbose = 0, save_best_only = True, mode = 'min') 
# callbacks_list = [two_opt_checkpoint]
# callbacks_list = [changer]
callbacks_list = [model_saving_checkpoint]
history = estimator_A.fit(X.T, Mask.T, validation_data=(X_valid.T, Mask_valid.T), epochs = 10, shuffle = True, batch_size=batch_size, steps_per_epoch=None, verbose = 1, callbacks= callbacks_list)
toc_adam = time.time()
# estimator_A.save_weights('model_weights.h5')
print("---------------------------second part of training---------------------------")
estimator_A.optimizer = sgd
# estimator_A.load_weights('model_weights.h5')
history1 = estimator_A.fit(X.T, Mask.T, validation_data=(X_valid.T, Mask_valid.T), epochs=10, shuffle = True, batch_size=batch_size, steps_per_epoch=None, verbose = 1, callbacks= callbacks_list) 
# new_model = load_model(two_opt_filepath)
# two_opt_checkpoint = ModelCheckpoint(two_opt_filepath, monitor = 'loss', verbose=1, save_best_only=True, mode='min')
# callbacks_list = [two_opt_checkpoint]
# new_model.optimizer = optimizers.SGD(lr=0.00001, decay=1e-6, momentum=0.9, nesterov=True)
# history_sgd = new_model.fit(X.T, Mask.T, shuffle = True, epochs = 10, batch_size=10,  steps_per_epoch=None, callbacks=callbacks_list)
# estimator_A.fit (X.T, y.T, shuffle = True, batch_size=100,  steps_per_epoch=None, callbacks = [changer])
# prediction = estimator_A.predict(X_test)

toc_fit = time.time()

^^^^^^^^^^^^^^^^^^^^^ Training on mixed5^^^^^^^^^^^^^^^^^^^^^
Train on 100 samples, validate on 21494 samples
Epoch 1/10
100/100 [==============================] - 14s 140ms/step - loss: 0.1048 - val_loss: 0.2285
Epoch 2/10
100/100 [==============================] - 12s 120ms/step - loss: 0.0778 - val_loss: 0.1637
Epoch 3/10
100/100 [==============================] - 12s 118ms/step - loss: 0.0652 - val_loss: 0.1500
Epoch 4/10
100/100 [==============================] - 11s 114ms/step - loss: 0.0579 - val_loss: 0.1158
Epoch 5/10
100/100 [==============================] - 12s 117ms/step - loss: 0.0605 - val_loss: 0.1323
Epoch 6/10
100/100 [==============================] - 12s 121ms/step - loss: 0.0538 - val_loss: 0.1357
Epoch 7/10
100/100 [==============================] - 12s 120ms/step - loss: 0.0530 - val_loss: 0.1204
Epoch 8/10
100/100 [==============================] - 12s 118ms/step - loss: 0.0464 - val_loss: 0.1238
Epoch 9/10
100/100 [==============================] - 12s 117ms/st

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [45]:
# X, y = data_reading(0)
X_dim= 1025
y_dim = 
estimator_A = Sequential()
estimator_A.add(Dense(h_A[0], input_dim = X_dim, kernel_initializer=glorot_normal(seed=None)))
estimator_A.add(BatchNormalization())
# estimator_A.add(Activation('sigmoid'))
estimator_A.add(LeakyReLU(alpha=0.1))
estimator_A.add(Dense(h_A[1], kernel_initializer=glorot_normal(seed=None)))
estimator_A.add(BatchNormalization())
# estimator_A.add(Activation('sigmoid'))
estimator_A.add(LeakyReLU(alpha=0.1))
estimator_A.add(Dense(h_A[2], kernel_initializer=glorot_normal(seed=None)))
estimator_A.add(BatchNormalization())
# estimator_A.add(Activation('sigmoid'))
estimator_A.add(LeakyReLU(alpha=0.1))
estimator_A.add(Dense(y_dim, kernel_initializer=glorot_normal(seed=None)))
estimator_A.add(BatchNormalization())
# estimator_A.add(Activation('sigmoid'))
# Compile model
estimator_A.compile(loss='mean_squared_error', optimizer='adam')

In [48]:
h5f = h5py.File('TIMIT/Organized/concatenated/mixed'  + data_series + '.hdf5','r')
ftr_shape = h5f['mixed' + data_series].shape
h5f.close()
batch_size = 100
steps_per_epoch = int(ftr_shape[1]/(batch_size))

In [49]:
tic = time.time()
estimator_A.fit_generator(generator=da(batch_size, ftr_shape[1]),
                          steps_per_epoch=steps_per_epoch,
                          epochs=200,
                          verbose=1)
toc  =time.time()


Epoch 1/100
[222000  62000 155200 ... 243200 111800 136800]
1399/1399 [==============================] - 17s 12ms/step - loss: 0.0033
Epoch 2/100
1399/1399 [==============================] - 18s 13ms/step - loss: 0.0022
Epoch 3/100
1399/1399 [==============================] - 15s 11ms/step - loss: 0.0011
Epoch 4/100
1399/1399 [==============================] - 17s 12ms/step - loss: 5.3483e-04
Epoch 5/100
1399/1399 [==============================] - 16s 11ms/step - loss: 2.1462e-04
Epoch 6/100
1399/1399 [==============================] - 14s 10ms/step - loss: 1.2093e-04
Epoch 7/100
1399/1399 [==============================] - 17s 12ms/step - loss: 9.7555e-05
Epoch 8/100
1399/1399 [==============================] - 18s 13ms/step - loss: 8.9262e-05
Epoch 9/100
1399/1399 [==============================] - 14s 10ms/step - loss: 8.7509e-05
Epoch 10/100
1399/1399 [==============================] - 14s 10ms/step - loss: 8.7388e-05
Epoch 11/100
1399/1399 [==============================] - 14s 1

In [50]:
print('Time= ', toc - tic)
# print("tIME OF K-FOLD=", (toc_fold - tic)/60)
# print("tIME OF FIT=", (toc_fit - tic)/60)

Time=  1430.0593705177307


In [51]:
print((toc -tic)/60)

23.83432284196218


In [52]:
estimator_A.model.save('Models/Two_stage/trained_model14.h5')

/home/majid/anaconda3/envs/Myenv/lib/python3.6/site-packages/keras/engine/sequential.py:110: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


In [99]:
estimator_A.model.save_weights('Models/Two_stage/weights12.h5')

In [4]:
estimator_A = load_model('Models/Two_stage/TSP_model0.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [60]:
data_series = str(0)
test_point_start0 = 0
test_point_stop0 = 1000

In [61]:
h5f = h5py.File('TSP/Organized/concatenated/Train_TSP_mixed_FFT'  + data_series + '.hdf5','r')
test_input = h5f['Train_TSP_mixed_FFT' + data_series][:, test_point_start0 : test_point_stop0]
print(test_input.shape)
h5f.close()

(1025, 1000)


In [62]:
h5f = h5py.File('TSP/Organized/concatenated/Train_TSP_clean_FFTs'  + data_series + '.hdf5','r')
test_target = h5f['Train_TSP_clean_FFTs' + data_series][:, test_point_start0 : test_point_stop0]
print(test_target.shape)
h5f.close()

(2050, 1000)


In [63]:
h5f = h5py.File('TSP/Organized/concatenated/Train_TSP_mixed_phase'  + data_series + '.hdf5','r')
mixed_phase = h5f['Train_TSP_mixed_phase' + data_series][:, test_point_start0 : test_point_stop0]
print(mixed_phase.shape)
h5f.close()

(1025, 1000)


In [64]:
h5f = h5py.File('TSP/Organized/concatenated/Train_TSP_clean_phases'  + data_series + '.hdf5','r')
clean_phase = h5f['Train_TSP_clean_phases' + data_series][:, test_point_start0 : test_point_stop0]
print(clean_phase.shape)
h5f.close()

(2050, 1000)


In [65]:
norms = np.zeros(test_input.shape[1])
for i in range(test_input.shape[1]):
    norms[i] = np.linalg.norm(test_input[:,i])

In [66]:
# test_input = sklearn.preprocessing.normalize(test_input.T, norm='l2', axis=1, copy=True, return_norm=False)
# test_target = sklearn.preprocessing.normalize(test_target.T, norm='l2', axis=1, copy=True, return_norm=False)
# test_input = (test_input-np.mean(test_input))/(np.max(test_input) - np.min(test_input))
test_input = preprocessing.normalize(test_input, norm='l2', axis=0, copy=True)
# test_target = preprocessing.normalize(test_target, norm='l2', axis=0, copy=True)
# test_target = (test_target-np.mean(test_target))/(np.var(test_target))

In [67]:
prediction = estimator_A.predict(test_input.T)

In [68]:
test_input = np.multiply(test_input,norms)

In [69]:
prediction.shape

(1000, 2050)

In [71]:
prediction1 = np.multiply(test_input.T, prediction[:,:1025])
prediction2 = np.multiply(test_input.T, prediction[:,1025:])

In [72]:
fft_samples = 1025

In [73]:
s1 = reconstruct(prediction1, mixed_phase.T)
s2 = reconstruct(prediction2, mixed_phase.T)

In [74]:
clean_recon1 = reconstruct(test_target[0:fft_samples,:].T, clean_phase[0:fft_samples,:].T)
clean_recon2 = reconstruct(test_target[fft_samples:,:].T, clean_phase[fft_samples:,:].T)

In [75]:
mixed_recon = reconstruct(test_input.T, mixed_phase.T)

In [76]:
s1.shape

(511488,)

In [53]:
sd.play(mixed_recon, 48000)

In [77]:
st_recon1 = stoi(s1, clean_recon1, 48000, extended=False)
st_recon2 = stoi(s2, clean_recon2, 48000, extended=False)

In [78]:
st_mixed1 = stoi(mixed_recon, clean_recon1, 48000, extended=False)
st_mixed2 = stoi(mixed_recon, clean_recon2, 48000, extended=False)

In [79]:
print('STOI_recon1=', st_recon1)
print('STOI_recon2=', st_recon2)
print('STOI_mixed1=', st_mixed1)
print('STOI_mixed2=', st_mixed2)

STOI_recon1= 0.8515234364494826
STOI_recon2= 0.9014656872841098
STOI_mixed1= 0.665909195846394
STOI_mixed2= 0.5782600675256552


In [80]:
sdr_recon1, sir_recon1, sar_recon1, perm_recon1 = separation.bss_eval_sources(s1, clean_recon1, compute_permutation=True)
sdr_recon2, sir_recon2, sar_recon2, perm_recon2 = separation.bss_eval_sources(s2, clean_recon2, compute_permutation=True)

In [82]:
sdr_mixed1, sir_mixed1, sar_mixed1, perm_mixed1 = separation.bss_eval_sources(mixed_recon, clean_recon1, compute_permutation=True)
sdr_mixed2, sir_mixed2, sar_mixed2, perm_mixed2 = separation.bss_eval_sources(mixed_recon, clean_recon2, compute_permutation=True)

In [83]:
print('SDR_recon1=', sdr_recon1)
print('SDR_recon2=', sdr_recon2)
print('SDR_mixed1=', sdr_mixed1)
print('SDR_mixed2=', sdr_mixed2)

SDR_recon1= [10.74456058]
SDR_recon2= [8.59891451]
SDR_mixed1= [3.01337799]
SDR_mixed2= [-0.690974]


In [52]:
# 
sd.play(s1, 48000)

In [ ]:
sd.play(s2, 48000)

In [ ]:
sd.play(clean_recon2, 48000)

In [34]:
sd.play(clean_recon1, 48000)

In [68]:
h5f = h5py.File('TIMIT/Organized/concatenated/mixed4_shuffled.hdf5','r')
a = h5f['mixed4_shuffled'][:,1]
h5f.close()

In [69]:
for i in range(test_input.shape[1]):
    if np.array_equal(a, test_input[:,i]):
        print(i)

41603


In [66]:
type(a)

numpy.ndarray